In [1]:
!pip install wget
import os
import torch
import pandas as pd
import numpy as np
import torch
from torch import nn


ssh_config = """
Host github.com
  IdentityFile ~/.ssh/github.pem
  User davipeag
  StrictHostKeyChecking no
"""

if os.name == 'nt':
  base_path = ""
  REPO_DIR = "."
  STORE_DIR ="." 
  print("Windows")
else:
  print("Unix-like")
  REPO_DIR = "/tmp/HeartRateRegression"
  from google.colab import drive, auth
  drive.mount('/content/drive')
  GIT_PATH = "/content/drive/My\ Drive/deeplearning_project/github.pem"
  DATA_DIR = os.path.join(REPO_DIR, "repo")
  STORE_DIR ="/content/drive/My Drive/deeplearning_project/" 
  !mkdir ~/.ssh
  !cp -u {GIT_PATH} ~/.ssh/
  !chmod u=rw,g=,o= ~/.ssh/github.pem
  !echo "{ssh_config}" > ~/.ssh/config
  !chmod u=rw,g=,o= ~/.ssh/config
  ! (cd /tmp && git clone git@github.com:davipeag/HeartRateRegression.git)
  ! (cd {REPO_DIR} && git pull )
  import sys
  sys.path.append(REPO_DIR)

def git_pull():
  ! (cd {REPO_DIR} && git pull )

git_pull()

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=bb31e9eaa869f47c1547e49fbc89fdc620dc76a36ef55057b2de0cc1dc9f1fa9
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
Unix-like
Mounted at /content/drive
Cloning into 'HeartRateRegression'...
remote: Enumerating objects: 159, done.
remote: Counting objects: 100% (159/159), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 1522 (delta 94), reused 93 (delta 41), pack-reused 1363
Receiving objects: 100% (1522/1522), 103.46 MiB | 22.71 MiB/s, done.
Resolving deltas: 100% (993/993), done.
Already up to date.
Already up to date.


In [2]:
import random
import torch
from torch import nn
import numpy as np
args = dict()
if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

SEED = 1234
def reset_seeds():
  random.seed(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  torch.cuda.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

# reset_seeds()

cpu


In [3]:
from data_utils import (Pamap2Handler, FormatPamap)

SUBJECTS = list(range(1,9))

handler = Pamap2Handler(DATA_DIR)
formatter = FormatPamap()
dfs_train = [formatter.transform(handler.get_protocol_subject(i)) for i in SUBJECTS]
[len(df)//200 for df in dfs_train]

download


[1882, 2235, 1264, 1647, 1873, 1809, 1567, 2040]

In [5]:
git_pull()

import importlib

import PPG
import preprocessing_utils
from PPG import FullTrainer

import RegressionHR
importlib.reload(RegressionHR)
from RegressionHR import Preprocessing
from RegressionHR import FullTrainer
from RegressionHR import PceLstmDefaults
from RegressionHR import PceLstmModel
from RegressionHR import TrainerJoint
from RegressionHR import  UtilitiesData
from RegressionHR import *

import Models

importlib.reload(Models.BaseModels)

# importlib.reload(PPG.AttentionDefaults)
importlib.reload(PPG)
# importlib.reload(PPG.UtilitiesDataXY)
importlib.reload(PPG.Models)
# importlib.reload(PPG.NoHrPceLstmModel)
# importlib.reload(PPG.TrainerXY)
# importlib.reload(PPG.TrainerIS)
# importlib.reload(PPG.FullTrainer)
# importlib.reload(PceLstmDefaults)
# importlib.reload(preprocessing_utils)

importlib.reload(RegressionHR.FullTrainer)
importlib.reload(RegressionHR.PceLstmDefaults)
importlib.reload(PPG.UtilitiesDataXY)
importlib.reload(preprocessing_utils)
importlib.reload(RegressionHR.TrainerJoint)
importlib.reload(RegressionHR.UtilitiesData)
importlib.reload(RegressionHR.PceLstmModel)
importlib.reload(preprocessing_utils)
# import imp
# for module in sys.modules.values():
#     importlib.reload(module)



Already up to date.


<module 'preprocessing_utils' from '/tmp/HeartRateRegression/preprocessing_utils.py'>

In [ ]:
import random 

options = {
  'ts_sub': [5],
  'batch_size': [64],
  'weight_decay': [0.0001, 0.00001, 0.0],
  'lr': [0.001, 0.005, 0.0001, 0.0005],
  'ts_per_sample':[30, 40, 50],
  "layer_sizes": [(4,4,4), (8,8,8), (16, 16, 16), (32,32,32), (64,64,64)],
  "skip_mapping": [((0,2),(0,3),(1,3)), ((0,2),(0,3)), ((1,3),(0,3)),  ((0,3),)],
  "dropout_rate": [0, 0.1, 0.2, 0.3, 0.4, 0.5]
  }


from  Optimization.Optimizers import RandomSearch
import json
import pickle

filename = "pamap_ffnn_discriminator_results.pkl"
save_path = os.path.join(STORE_DIR, filename)

try:
  with open(save_path, "rb") as f:
    results = pickle.load(f)
except FileNotFoundError:
  results = list()

full_trainer = RegressionHR.FullTrainer.IteractiveFFNNFullTrainerJointValidation(dfs_train, args["device"], nepoch =  500)

searcher = RandomSearch(full_trainer, options)
searcher.results = results
while True:
  searcher.fit(1)
  with open(save_path, "wb") as f:
    pickle.dump(searcher.results, f)

best val epoch: 1
[1/500]: loss_train: 20.493 loss_val 21.462 loss_ts 21.557
best val epoch: 2
[2/500]: loss_train: 20.043 loss_val 21.197 loss_ts 21.197
best val epoch: 3
[3/500]: loss_train: 19.660 loss_val 21.032 loss_ts 20.896
best val epoch: 4
[4/500]: loss_train: 19.338 loss_val 20.909 loss_ts 20.608
best val epoch: 5
[5/500]: loss_train: 19.051 loss_val 20.848 loss_ts 20.189
best val epoch: 6
[6/500]: loss_train: 18.848 loss_val 20.802 loss_ts 19.989
best val epoch: 7
[7/500]: loss_train: 18.649 loss_val 20.623 loss_ts 19.954
best val epoch: 8
[8/500]: loss_train: 18.399 loss_val 20.289 loss_ts 20.132
best val epoch: 9
[9/500]: loss_train: 18.219 loss_val 20.015 loss_ts 20.368
best val epoch: 10
[10/500]: loss_train: 18.013 loss_val 19.810 loss_ts 20.384
best val epoch: 11
[11/500]: loss_train: 17.800 loss_val 19.617 loss_ts 20.287
best val epoch: 12
[12/500]: loss_train: 17.559 loss_val 19.369 loss_ts 20.233
best val epoch: 13
[13/500]: loss_train: 17.319 loss_val 19.126 loss_t